In [1]:
import mne
import os
from ieeg.viz.mri import plot_subj, subject_to_info, plot_on_average
from ieeg.io import raw_from_layout, get_data
from misc_functions import load_sig_chans, channel_names_to_indices

### plot single subject's significant electrodes

In [10]:
subj = "D0057"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)

subj_without_zeros = "D" + subj[1:].lstrip('0')
plot_subj(subj_without_zeros, picks=stim_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Stimulus', color=(1, 1, 0)) 
plot_subj(subj_without_zeros, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', color=(1, 1, 0))



Loaded significant channels for subject D0057
Loaded significant channels for subject D0057


c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:547: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:547: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


### plot all significant electrodes on average brain

In [3]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary

for subj in subjects:
    if subj == "D0077":
        continue
    
    resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'

    resp_sig_chans = load_sig_chans(resp_filename)
    stim_sig_chans = load_sig_chans(stim_filename)

    # Combine the significant electrodes for this subject
    combined_chans = resp_sig_chans + stim_sig_chans

    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans


plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0))

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24})
Loaded significant channels for subject D0057
Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0071


c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:567: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:567: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:567: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:567: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:567: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
c:\Users\jz421\Desktop\GlobalLocal\

In [8]:
subject_to_info('D57')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jz421\\Box\\CoganLab\\ECoG_Recon_Full\\D57\\elec_recon\\D57_elec_locations_RAS_brainshifted.txt'